In [ ]:
import cv2
import numpy as np
from keras.models import load_model

model = load_model(r"screw_bolt_model.h5", compile=False)
class_names = [line.strip() for line in open("labels.txt", "r").readlines()]

image_path = r"C:\Users\Admin\Downloads\ScrewAndBolt_20240713-20240731T091427Z-001\ScrewAndBolt_20240713\20240713_194606.jpg"
image_read = cv2.imread(image_path)

gray = cv2.cvtColor(image_read, cv2.COLOR_BGR2GRAY)

blur = cv2.GaussianBlur(gray, (3, 3), 0)

_, threshold = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY_INV)

contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

screw_count = 0
bolt_count = 0
min_contour_area = 300

for contour in contours:
    area = cv2.contourArea(contour)
    if area > min_contour_area:
        x, y, w, h = cv2.boundingRect(contour)

        roi = image_read[y:y+h, x:x+w]

        resize_roi = cv2.resize(roi, (224, 224))
        roi_array = np.asarray(resize_roi, dtype=np.float32).reshape(1, 224, 224, 3)
        array_roi = (roi_array / 127.5) - 1

        prediction = model.predict(array_roi)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        print(f"Class: {class_name}, Confidence Score: {confidence_score}")

        if 'screw' in class_name and confidence_score > 0.5:  # Adjust the confidence threshold as needed
            screw_count += 1
            label = "Screw"
            color = (0, 255, 0)
        elif 'bolt' in class_name and confidence_score > 0.5:  # Adjust the confidence threshold as needed
            bolt_count += 1
            label = "Bolt"
            color = (255, 0, 0)
        else:
            label = "Unknown"
            color = (0, 0, 255)

        cv2.rectangle(image_read, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image_read, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


print(f"Number of screws detected: {screw_count}")
print(f"Number of bolts detected: {bolt_count}")

image = cv2.resize(image_read, (700, 700))
cv2.imshow("Detected Screws and Bolts", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
